In [ ]:
clear all;

global ipynb = 'hand-digits-conv2D';

source('clearest-nn.m');
source('utils-logging.m');
source('utils-training.m');

log2file(tmp('log'));

##########################################

data = load('datasets/train_signs.h5');

classes = data.list_classes;
printvar('classes');

X1 = data.train_set_x;
Y1 = data.train_set_y;
printsize('X1');
printsize('Y1');

data = load('datasets/test_signs.h5');

X2 = data.test_set_x;
Y2 = data.test_set_y;
printsize('X2');
printsize('Y2');

function [X, Y, Yoh] = PREPARE(X, Y, classes)
    lim = max(classes) + 1;
    idx = find(Y < lim);
    X   = slice(X, [], idx);
    Y   = slice(Y, [], idx);
    X   = reshape(X, 64, 64, 3, count(X));
    X   = double(X) / 255;
    Yoh = onehots(Y, classes);    
end

%classes = classes(find(classes < 3));
%printvar('classes');

[X1, Y1, Y1oh] = PREPARE(X1, Y1, classes);
printsize('X1');
printsize('Y1oh');

[X2, Y2, Y2oh] = PREPARE(X2, Y2, classes);
printsize('X2');
printsize('Y2oh');

[X1, Y1, Y1oh] = shuffle(X1, Y1, Y1oh);
[X2, Y2, Y2oh] = shuffle(X2, Y2, Y2oh);

oh1 = all(Y1 == onecolds(Y1oh, classes, -1))
oh2 = all(Y2 == onecolds(Y2oh, classes, -1))

BBB = minibatches(X1, Y1oh, 32);
printsize('BBB');
printsize('BBB{1}.X');
printsize('BBB{1}.Y');

function SHOW(x, y)
    x = reshape(x, 3, 64, 64, 1);
    x = permute(x, [2, 3, 1]);
    figure;
    imshow(x);
    title(disp(y));
end

function acc = TEST(CTX, X, Y, classes, alt)
    m   = count(X);
    Z   = predict(CTX, X);
    Z   = maxcolds(Z, classes, alt);
    acc = nnz(Z == Y) / m;
end

function TEST1(CTX, x, classes, alt)
    x(:,:,:,2) = x;
    z = predict(CTX, x);
    x = x(:,:,:,1);
    z = z(:,1);
    z = maxcolds(z, classes, alt);
    SHOW(x, z);
end

[x,y] = pick(X1, Y1);
SHOW(x,y);
#

In [ ]:
function CTX = LEARN(CTX, times=101, varargin)
    CTX           = TUNE(CTX, varargin{:});
    BBB           = evalin('caller', 'BBB');
    X2            = evalin('caller', 'X2');
    Y2            = evalin('caller', 'Y2');
    classes       = evalin('caller', 'classes');
    [CTX, ok]     = TRAIN(CTX, BBB, times); 
    test_accuracy = TEST(CTX, X2, Y2, classes, alt=-1);
    printvar('test_accuracy');
    global ipynb;
    notifed = pushnotify(sprintf('%s: smth_learnt=%s, test_accuracy=%f', ipynb, bool2yesno(ok), test_accuracy))
    if ok
        save('-binary', tmp('ctx'), 'CTX');
    end
end

In [ ]:
source('clearest-nn.m');
source('utils-logging.m');

rand('state', 1);

num_c = count(classes);
printvar('num_c');

CTX = model( [28,28,3]
           , {'conv2D', [14,14,6], 3:6}
           , 'relu'
           , {'conv2D', [7,7,12], 3:6}
           , 'relu'
           , 'reshape'
           , {'dense', num_c}
           , 'softmax'
           );
           
%load('-binary', tmp('ctx'), 'CTX');

CTX = optimization( CTX
                  , {'L2_regularization', 0.0001}
%                   , {'gradient_clipping', 0.5}      
%                  , {'descent', 0.001}
                  , {'momentum', 0.0001}
%                   , {'nesterov', 0.0001}
%                   , {'adam', 0.001}
%                   , 'rprop'
                  , {'gradient_clipping',0.9}
                  , 'stats'
                  );

CTX = objective(CTX, 'logloss');

printmodel('CTX');


In [ ]:
CTX = LEARN(CTX, times=101, {'momentum', 'rate', 0.001});

momentum.rate = 0.001000

starts_at = 12-May-2020 17:07:07
GRAD_DIFF = 0.951998
params_checked = 6

In [ ]:
CTX = LEARN(CTX, times=101, {'momentum', 'rate', 0.001});

In [ ]:
CTX = LEARN(CTX, times=101, {'momentum', 'rate', 0.001});

In [ ]:
CTX = LEARN(CTX, times=101, {'momentum', 'rate', 0.0005});

In [ ]:
#load('-binary', tmp('ctx'), 'CTX');


In [ ]:
# test_accuracy = TEST(CTX, X2, Y2, classes, alt=-1)

# TEST1(CTX, pick(X2), classes, alt=-1);


In [ ]:
% CTX = model(12288, {'dense', 25}, 'relu', {'dense', 12}, 'relu', {'dense', num_c}, 'softmax');

% NOW = 19-Apr-2020 15:34:09
% train_accuracy =  0.251388888888889

% 1000 0.067233

% train_accuracy =  0.956944444444444
% NOW = 19-Apr-2020 16:13:38
% test_accuracy =  0.862500000000000